Student 1: Elad_Natan:, i.d.: 308542141 , github: https://github.com/elad014/machine_learning.git
Student 2: Ofir Goldstein: , i.d.: 207930868 , github: https://github.com/ofir246/Machine_Lerning_RM.git
Student 3: name: , i.d.: , github:

1. Load breast cancer dataset (**structured data**)

For more details about the data: https://scikit-learn.org/1.5/modules/generated/sklearn.datasets.load_breast_cancer.html

In [1]:


from sklearn.datasets import load_breast_cancer
my_data = load_breast_cancer()



2. Split **my_data** to train and test:

- Define X_train, X_test, Y_train, Y_test
- Choose **test_size** for splitting **my_data**
- Use **train_test_split** (for details: https://scikit-learn.org/dev/modules/generated/sklearn.model_selection.train_test_split.html)

In [2]:

from sklearn.model_selection import train_test_split

my_data = load_breast_cancer()
X = my_data.data
y = my_data.target

X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    y,
    test_size=0.33,
    random_state=42
)

print(X_train)
print(X_test)
print(Y_train)
print(Y_test)

# X_train, X_test, Y_train, Y_test = train_test_split(...)

[[1.546e+01 1.189e+01 1.025e+02 ... 1.827e-01 3.216e-01 1.010e-01]
 [1.285e+01 2.137e+01 8.263e+01 ... 5.601e-02 2.488e-01 8.151e-02]
 [1.921e+01 1.857e+01 1.255e+02 ... 2.091e-01 3.537e-01 8.294e-02]
 ...
 [1.429e+01 1.682e+01 9.030e+01 ... 3.333e-02 2.458e-01 6.120e-02]
 [1.398e+01 1.962e+01 9.112e+01 ... 1.827e-01 3.179e-01 1.055e-01]
 [1.218e+01 2.052e+01 7.722e+01 ... 7.431e-02 2.694e-01 6.878e-02]]
[[1.247e+01 1.860e+01 8.109e+01 ... 1.015e-01 3.014e-01 8.750e-02]
 [1.894e+01 2.131e+01 1.236e+02 ... 1.789e-01 2.551e-01 6.589e-02]
 [1.546e+01 1.948e+01 1.017e+02 ... 1.514e-01 2.837e-01 8.019e-02]
 ...
 [1.371e+01 2.083e+01 9.020e+01 ... 1.556e-01 3.196e-01 1.151e-01]
 [1.447e+01 2.499e+01 9.581e+01 ... 1.205e-01 3.187e-01 1.023e-01]
 [1.354e+01 1.436e+01 8.746e+01 ... 1.288e-01 2.977e-01 7.259e-02]]
[0 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 0 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 1
 1 0 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 1 0 1 1 1 0 1 0 1 0 1 0
 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 1 1

3. Libraries

In [ ]:
!pip install mlflow
!pip install mlflow scikit-learn

import mlflow
import mlflow.sklearn
from mlflow import log_param, log_metric

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import itertools
import pandas as pd


4. Define MLFlow experiment

In [ ]:
EXPERIMENT_NAME = "trees_hyperparam"
mlflow.set_experiment(EXPERIMENT_NAME)
# MLFlow details: https://mlflow.org/docs/latest/ml/tracking/

5. Train **model_decision_tree**

- Library: sklearn.tree.DecisionTreeClassifier
- Data: X_train, Y_train
- **Essential**: explore and optimize DecisionTreeClassifier options   

In [ ]:
from sklearn.tree import DecisionTreeClassifier

max_depth_list                = [1, 3, 30, 50, 70, 100, 150]
min_samples_split_list        = [2, 4, 25, 60, 75, 120, 200]
criterion_list                = ["gini", "entropy"]
splitter_list                 = ["best", "random"]

param_grid = list(itertools.product(
    max_depth_list,
    min_samples_split_list,
    criterion_list,
    splitter_list,
    ))

for max_depth,\
    min_samples_split,\
    criterion,\
    splitter,\
    in param_grid:
    with mlflow.start_run():

        # Log parameters
        mlflow.log_param("model_type", "DecisionTree")
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("min_samples_split", min_samples_split)
        mlflow.log_param("criterion", criterion)
        mlflow.log_param("splitter", splitter)

        # Train the model
        d_tree = DecisionTreeClassifier(
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            criterion=criterion,
            splitter=splitter,
            )
        d_tree.fit(X_train, Y_train) #actual_train

        # run test prediction and calculate metrics:
        # accuracy_score
        # precision_score
        # recall_score
        # f1_score

        y_pred = d_tree.predict(X_test)

        acc = accuracy_score(Y_test, y_pred)
        pre = precision_score(Y_test, y_pred)
        rec = recall_score(Y_test, y_pred)
        f1 = f1_score(Y_test, y_pred)

        # Log metrics
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision_score", pre)
        mlflow.log_metric("recall_score", rec)
        mlflow.log_metric("f1_score", f1)

6. Train model_random_forest
- Library: sklearn.ensemble.RandomForestClassifier
- Data: X_train, Y_train
- **Essential**: explore and optimize RandomForestClassifier options

In [ ]:
from sklearn.ensemble import RandomForestClassifier

n_estimators_list             = [10, 50, 100, 150, 200]
max_depth_list                = [1, 3, 30, 50, 70, 100, 150]
min_samples_split_list        = [2, 4, 25, 60, 75, 120, 200]
criterion_list                = ["gini", "entropy", "log_loss"]

param_grid = list(itertools.product(
    n_estimators_list,
    max_depth_list,
    min_samples_split_list,
    criterion_list,
))

for n_estimators,\
    max_depth,\
    min_samples_split,\
    criterion,\
    in param_grid:
    with mlflow.start_run():

        # Log parameters
        mlflow.log_param("model_type", "RandomForest")
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("min_samples_split", min_samples_split)
        mlflow.log_param("criterion", criterion)

        # Train the model
        rf = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            criterion=criterion,
        )
        rf.fit(X_train, Y_train)

        # run test prediction and calculate metrics:
        # accuracy_score
        # precision_score
        # recall_score
        # f1_score
        y_pred = rf.predict(X_test)

        acc = accuracy_score(Y_test, y_pred)
        pre = precision_score(Y_test, y_pred)
        rec = recall_score(Y_test, y_pred)
        f1 = f1_score(Y_test, y_pred)

        # Log metrics
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision_score", pre)
        mlflow.log_metric("recall_score", rec)
        mlflow.log_metric("f1_score", f1)




7. Train model_adaboost

- Library: sklearn.ensemble.AdaBoostClassifier
- Data: X_train, Y_train
- **Essential**: explore and optimize AdaBoostClassifier options

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

n_estimators_list  = [10, 30, 50, 70, 100, 150, 200]
learning_rate_list = [0.01, 0.1, 0.5, 1.0, 1.5, 2.0]
random_state_list  = [42, 123, 456, 789, 101112]

param_grid = list(itertools.product(
    n_estimators_list,
    learning_rate_list,
    random_state_list
    ))

for n_estimators, learning_rate, random_state in param_grid:
    with mlflow.start_run():

        # Log parameters
        mlflow.log_param("model_type", "AdaBoost")
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("random_state", random_state)

        # Train the model
        ada = AdaBoostClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            random_state=random_state
            )
        ada.fit(X_train, Y_train)

        # run test prediction and calculate metrics:
        # accuracy_score
        # precision_score
        # recall_score
        # f1_score
        y_pred = ada.predict(X_test)

        acc = accuracy_score(Y_test, y_pred)
        pre = precision_score(Y_test, y_pred)
        rec = recall_score(Y_test, y_pred)
        f1 = f1_score(Y_test, y_pred)

        # Log metrics
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision_score", pre)
        mlflow.log_metric("recall_score", rec)
        mlflow.log_metric("f1_score", f1)



8. Store the result

In [ ]:
from google.colab import files

df = mlflow.search_runs(experiment_names=[EXPERIMENT_NAME])

df = df.drop(columns=[col for col in df.columns if "time" in col.lower()], errors="ignore")

df.to_excel("elad_natan_ofir_goldstine.xlsx", index=False)

files.download("elad_natan_ofir_goldstine.xlsx")
